# Model 2 - UDE Framework

**System:**
$$\frac{dX}{dt} = f(Y) - d \cdot X$$
$$\frac{dY}{dt} = X - d \cdot Y$$

**Unknown:** Function $f(Y)$  
**Unknown Parameter:** $d$


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from ude_framework import (
    DataGenerator, 
    create_ode_system,
    create_neural_network,
    create_ude,
    UDETrainer,
    UDEEvaluator
)

%matplotlib inline
np.random.seed(42)
torch.manual_seed(42)

print("✓ UDE Framework loaded!")


ModuleNotFoundError: No module named 'ude_framework'

In [ ]:
# ============================================================================
# CONFIGURE EVERYTHING HERE
# ============================================================================

# ----------------------------------------------------------------------------
# 1. TRUE ODE SYSTEM (for generating data)
# ----------------------------------------------------------------------------
def true_ode_equations(t, y, params):
    """
    Model 2 TRUE system:
    dX/dt = f(Y) - d*X
    dY/dt = X - d*Y
    
    Here we assume f(Y) is a Hill function
    """
    X, Y = y
    v, K, n, d = params['v'], params['K'], params['n'], params['d']
    
    # True f(Y): Hill activation
    f_Y = v * (Y**n) / (K**n + Y**n)
    
    dX_dt = f_Y - d * X
    dY_dt = X - d * Y
    
    return [dX_dt, dY_dt]

# True system parameters
TRUE_PARAMS = {
    'v': 8.0,    # max rate
    'K': 1.5,    # Hill constant
    'n': 2.0,    # Hill coefficient
    'd': 0.8     # degradation rate (UNKNOWN in UDE)
}

# State variable names
STATE_NAMES = ['X', 'Y']

# Initial conditions
INITIAL_CONDITIONS = np.array([2.0, 0.5])

# Time settings
T_START = 0.0
T_END = 30.0
N_POINTS = 800

# Noise settings
NOISE_LEVEL = 0.05
NOISE_TYPE = 'relative'

# ----------------------------------------------------------------------------
# 2. UDE STRUCTURE
# ----------------------------------------------------------------------------
def ude_ode_equations(t, y, nn_outputs, known_params):
    """
    UDE for Model 2:
    - KNOWN: linear structure
    - UNKNOWN: f(Y) learned by NN
    - LEARNABLE PARAMETER: d
    """
    X = y[..., 0:1]
    Y = y[..., 1:2]
    
    f_Y_nn = nn_outputs['f_Y_nn']
    if f_Y_nn.dim() > 2:
        f_Y_nn = f_Y_nn.squeeze(-1)
    
    d = known_params.get('d_learnable', torch.tensor(0.8))
    
    dX_dt = f_Y_nn - d * X
    dY_dt = X - d * Y
    
    return torch.cat([dX_dt, dY_dt], dim=-1)

KNOWN_PARAMS = {}

# ----------------------------------------------------------------------------
# 3. NEURAL NETWORK CONFIGURATION
# ----------------------------------------------------------------------------
NN_INPUT_DIM = 1   # f depends on Y
NN_OUTPUT_DIM = 1  # outputs f(Y)

def nn_input_extractor(y):
    """Extract Y (second state) for f(Y)"""
    return y[..., 1:2]  # Returns Y

NN_ARCHITECTURE = 'flexible'
NN_CONFIG = {
    'hidden_dims': [64, 64, 64],
    'activation': 'tanh',
    'final_activation': 'softplus',
    'use_batch_norm': False,
    'dropout': 0.0
}

NN_NAME = 'f_Y_nn'

# ----------------------------------------------------------------------------
# 4. TRAINING CONFIGURATION
# ----------------------------------------------------------------------------
N_EPOCHS = 500
LEARNING_RATE = 1e-3
OPTIMIZER = 'adam'
WEIGHT_DECAY = 0.0
GRAD_CLIP = 1.0

SCHEDULER_TYPE = 'plateau'
SCHEDULER_PARAMS = {
    'factor': 0.5,
    'patience': 80,
    'min_lr': 1e-6,
    'verbose': False
}

ODE_SOLVER = 'dopri5'
ODE_RTOL = 1e-6
ODE_ATOL = 1e-8

LOSS_TYPE = 'mse'
STATE_WEIGHTS = None
PRINT_EVERY = 20

# ----------------------------------------------------------------------------
# 5. OPTIONAL: TRUE FUNCTION
# ----------------------------------------------------------------------------
def true_function_for_comparison(nn_input):
    """True f(Y) = Hill function"""
    Y = nn_input
    v, K, n = TRUE_PARAMS['v'], TRUE_PARAMS['K'], TRUE_PARAMS['n']
    return v * (Y**n) / (K**n + Y**n)

FUNCTION_INPUT_RANGE = np.linspace(0, 5, 300)
FUNCTION_INPUT_NAME = 'Y Concentration'
FUNCTION_OUTPUT_NAME = 'f(Y)'

print("✓ Configuration complete!")


## 1. Generate Data


In [ ]:
true_system = create_ode_system(
    name="Model 2 True System",
    equations=true_ode_equations,
    params=TRUE_PARAMS,
    state_names=STATE_NAMES
)

data_gen = DataGenerator(true_system)
data = data_gen.generate(
    initial_conditions=INITIAL_CONDITIONS,
    t_span=(T_START, T_END),
    n_points=N_POINTS,
    noise_level=NOISE_LEVEL,
    noise_type=NOISE_TYPE,
    random_seed=42
)

t = data['t']
y_true = data['y_true']
y_noisy = data['y_noisy']
y0 = data['y0']

print(f"✓ Generated {len(t)} points from t={T_START} to {T_END}")
print(f"  States: {STATE_NAMES}")
print(f"  Noise: {NOISE_LEVEL*100}% {NOISE_TYPE}")


In [ ]:
n_states = len(STATE_NAMES)
fig, axes = plt.subplots(n_states, 1, figsize=(12, 3*n_states))
if n_states == 1:
    axes = [axes]

for i, (ax, name) in enumerate(zip(axes, STATE_NAMES)):
    ax.plot(t, y_true[:, i], 'b-', label=f'True', linewidth=2)
    ax.plot(t, y_noisy[:, i], 'r.', label=f'Noisy', alpha=0.5, markersize=2)
    ax.set_ylabel(name, fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel('Time', fontsize=12)
plt.suptitle('Model 2: Generated Data', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Phase portrait
plt.figure(figsize=(8, 6))
plt.plot(y_true[:, 0], y_true[:, 1], 'b-', linewidth=2, label='True')
plt.plot(y_noisy[:, 0], y_noisy[:, 1], 'r.', alpha=0.3, markersize=2, label='Noisy')
plt.plot(y0[0], y0[1], 'go', markersize=10, label='IC', zorder=5)
plt.xlabel(STATE_NAMES[0], fontsize=12)
plt.ylabel(STATE_NAMES[1], fontsize=12)
plt.title('Phase Portrait', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


## 2. Build UDE Model


In [ ]:
nn_model = create_neural_network(
    input_dim=NN_INPUT_DIM,
    output_dim=NN_OUTPUT_DIM,
    architecture=NN_ARCHITECTURE,
    **NN_CONFIG
)

print("Neural Network:")
print(nn_model)
print(f"\nParameters: {nn_model.count_parameters():,}")


In [ ]:
class NNWrapper(nn.Module):
    def __init__(self, nn, input_extractor):
        super().__init__()
        self.nn = nn
        self.input_extractor = input_extractor
    
    def forward(self, t, y):
        nn_input = self.input_extractor(y)
        return self.nn(t, nn_input)

wrapped_nn = NNWrapper(nn_model, nn_input_extractor)

d_learnable = nn.Parameter(torch.tensor(1.0))  # Initial guess
KNOWN_PARAMS['d_learnable'] = d_learnable

ude_model = create_ude(
    n_states=len(STATE_NAMES),
    ode_func=ude_ode_equations,
    neural_networks={NN_NAME: wrapped_nn},
    known_params=KNOWN_PARAMS
)

ude_model.register_parameter('d_learnable', d_learnable)

print(f"\n✓ UDE Model created!")
print(f"  NN parameters: {ude_model.count_parameters()['total']}")
print(f"  Initial d: {d_learnable.item():.4f} (True: {TRUE_PARAMS['d']})")


## 3. Train UDE


In [ ]:
t_torch = torch.tensor(t, dtype=torch.float32)
y_noisy_torch = torch.tensor(y_noisy, dtype=torch.float32)
y_true_torch = torch.tensor(y_true, dtype=torch.float32)
y0_torch = torch.tensor(y0, dtype=torch.float32)

trainer = UDETrainer(
    ude_model=ude_model,
    optimizer_name=OPTIMIZER,
    learning_rate=LEARNING_RATE,
    scheduler_type=SCHEDULER_TYPE,
    scheduler_params=SCHEDULER_PARAMS,
    weight_decay=WEIGHT_DECAY,
    grad_clip=GRAD_CLIP,
    ode_solver=ODE_SOLVER,
    ode_rtol=ODE_RTOL,
    ode_atol=ODE_ATOL
)

print("✓ Trainer initialized")


In [ ]:
print(f"\n{'='*60}")
print(f"TRAINING ({N_EPOCHS} epochs)")
print(f"{'='*60}\n")

trainer.train(
    y0=y0_torch,
    t=t_torch,
    y_true=y_noisy_torch,
    n_epochs=N_EPOCHS,
    weights=STATE_WEIGHTS,
    loss_type=LOSS_TYPE,
    print_every=PRINT_EVERY
)

print(f"\nLearned d: {ude_model.d_learnable.item():.4f} (True: {TRUE_PARAMS['d']})")


In [ ]:
history = trainer.get_history()
evaluator = UDEEvaluator(ude_model, STATE_NAMES)
evaluator.plot_training_history(history)


## 4. Evaluate UDE Performance


In [ ]:
with torch.no_grad():
    y_pred = trainer.forward_solve(y0_torch, t_torch).numpy()

metrics = evaluator.compute_metrics(y_pred, y_true)
evaluator.print_metrics(metrics)


In [ ]:
evaluator.plot_trajectories(
    t=t,
    y_pred=y_pred,
    y_true=y_true,
    y_noisy=y_noisy
)


In [ ]:
evaluator.plot_phase_portrait(
    y_pred=y_pred,
    y_true=y_true,
    y0=y0,
    y_noisy=y_noisy,
    state_indices=(0, 1)
)


## 5. Compare Learned NN vs True Function


In [ ]:
evaluator.compare_learned_function(
    nn_model=nn_model,
    true_function=true_function_for_comparison,
    input_range=FUNCTION_INPUT_RANGE,
    input_name=FUNCTION_INPUT_NAME,
    output_name=FUNCTION_OUTPUT_NAME
)


## 6. Parameter Comparison


In [ ]:
print("\n" + "="*60)
print("PARAMETER COMPARISON")
print("="*60)
print(f"\nParameter d:")
print(f"  True value:    {TRUE_PARAMS['d']:.4f}")
print(f"  Learned value: {ude_model.d_learnable.item():.4f}")
print(f"  Error:         {abs(ude_model.d_learnable.item() - TRUE_PARAMS['d']):.4f}")
print(f"  Relative error: {abs(ude_model.d_learnable.item() - TRUE_PARAMS['d'])/TRUE_PARAMS['d']*100:.2f}%")
print("="*60)
